![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

# Getting Started with Iceberg and Spark

In [1]:
spark

## Load One Month of NYC Taxi/Limousine Trip Data

For this notebook, we will use the New York City Taxi and Limousine Commision Trip Record Data that's available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. We'll save this into an iceberg table called `taxis`.

In [2]:
df = spark.read.option("header", True).csv("/home/iceberg/data/yello_tripdata_2020-04.csv")
df.write.saveAsTable("taxis")

In [3]:
%%sql

DESCRIBE TABLE taxis

col_name,data_type,comment
VendorID,string,
tpep_pickup_datetime,string,
tpep_dropoff_datetime,string,
passenger_count,string,
trip_distance,string,
RatecodeID,string,
store_and_fwd_flag,string,
PULocationID,string,
DOLocationID,string,
payment_type,string,


In [4]:
%%sql

SELECT COUNT(*) as cnt
FROM taxis

cnt
237993


## Schema Evolution

Adding, dropping, renaming, or altering columns is easy and safe in Iceberg. In this example, we'll rename `fare_amount` to `fare` and `trip_distance` to `distance`. We'll also add a float column `fare_per_distance_unit` immediately after `distance`.

In [5]:
%%sql

ALTER TABLE taxis
RENAME COLUMN fare_amount TO fare

++
||
++
++

In [6]:
%%sql

ALTER TABLE taxis
RENAME COLUMN trip_distance TO distance

++
||
++
++

In [7]:
%%sql

ALTER TABLE taxis
ADD COLUMN fare_per_distance_unit float AFTER distance

++
||
++
++

Let's update the new `fare_per_distance_unit` to equal `fare` divided by `distance`.

In [8]:
%%sql

UPDATE taxis
SET fare_per_distance_unit = fare/distance

++
||
++
++

In [9]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,fare,distance,fare_per_distance_unit
1,2020-04-01 00:41:22,2020-04-01 01:01:53,5.5,1.20,4.583333492279053
1,2020-04-01 00:56:00,2020-04-01 01:09:25,12.5,3.40,3.6764705181121826
1,2020-04-01 00:00:26,2020-04-01 00:09:25,10,2.80,3.5714285373687744
1,2020-04-01 00:24:38,2020-04-01 00:34:38,10,2.60,3.846153736114502
2,2020-04-01 00:13:24,2020-04-01 00:18:26,6.5,1.44,4.513888835906982
2,2020-04-01 00:24:36,2020-04-01 00:33:09,10.5,2.93,3.583617687225342
2,2020-04-01 00:56:56,2020-04-01 01:09:13,20,6.86,2.915452003479004
2,2020-04-01 00:06:56,2020-04-01 00:14:15,7,1.19,5.882352828979492
1,2020-04-01 00:50:05,2020-04-01 01:08:54,31.5,11.30,2.7876105308532715
2,2020-04-01 00:07:10,2020-04-01 00:18:45,13,3.68,3.5326087474823


## Row Level Deletes
With Iceberg tables, `DELETE` queries can be used to perform row-level deletes. This is as simple as providing the table name and a `WHERE` predicate. If the filter matches an entire partition of the table, Iceberg will intelligently perform a metadata-only operation where it simply deletes the metadata for that partition.

Let's perform a row-level delete for all rows that have a `fare_per_distance_unit` greater than 4 and a `distance` greater than 2. This should leave us with relatively short trips that have a relatively high fare per distance traveled.

In [10]:
%%sql

DELETE FROM taxis
WHERE fare_per_distance_unit > 4.0
OR distance > 2.0

21/11/19 18:31:48 WARN RewriteDelete: Cannot translate expression to source filter: ((fare_per_distance_unit#289 > 4.0) OR (cast(distance#288 as double) > 2.0))


++
||
++
++

There are some fares that have a `null` for `fare_per_distance_unit` due to the distance being `0`. Let's remove those as well.

In [11]:
%%sql

DELETE FROM taxis
WHERE fare_per_distance_unit is null

++
||
++
++

In [12]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,fare,distance,fare_per_distance_unit
2,2020-04-01 00:42:56,2020-04-01 00:48:41,-7,1.34,-5.223880767822266
2,2020-04-01 00:26:49,2020-04-01 00:31:04,6.5,1.69,3.846153736114502
2,2020-04-01 00:42:30,2020-04-01 00:46:58,6.5,1.70,3.8235294818878174
1,2020-04-01 00:36:15,2020-04-01 00:41:30,7,1.80,3.8888888359069824
2,2020-04-01 01:16:34,2020-04-01 01:23:20,-8,1.85,-4.324324131011963
2,2020-04-01 01:51:36,2020-04-01 01:57:59,7.5,1.88,3.9893617630004883
1,2020-04-01 02:13:27,2020-04-01 02:16:30,6,1.50,4.0
2,2020-04-01 02:25:38,2020-04-01 02:26:05,-2.5,.08,-31.25
2,2020-04-01 02:29:54,2020-04-01 02:33:27,6,1.52,3.9473683834075928
2,2020-04-01 02:39:47,2020-04-01 02:45:27,-8,1.94,-4.123711109161377


In [13]:
%%sql

SELECT COUNT(*) as cnt
FROM taxis

cnt
4953


## Metadata Tables

Iceberg tables contain very rich metadata that can be easily queried. For example, you can retrieve the manifest list for any snapshot, simply by querying the table's `snapshots` table.

In [14]:
%%sql

SELECT snapshot_id, manifest_list
FROM taxis.snapshots

21/11/19 18:31:51 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/11/19 18:31:51 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/11/19 18:31:53 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
21/11/19 18:31:53 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.22.0.3
21/11/19 18:31:53 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


snapshot_id,manifest_list
5166552351258945488,/home/iceberg/warehouse/taxis/metadata/snap-5166552351258945488-1-6be84df9-303b-463b-8dad-3aae418c0c46.avro
4397789499468740865,/home/iceberg/warehouse/taxis/metadata/snap-4397789499468740865-1-014a8d76-336a-4945-be5b-4f0b651bac84.avro
7769498799260430115,/home/iceberg/warehouse/taxis/metadata/snap-7769498799260430115-1-8f976b8c-5d07-4a6d-974e-ae346e3b03bd.avro
6092828293901762509,/home/iceberg/warehouse/taxis/metadata/snap-6092828293901762509-1-14378dc8-29c9-4df7-a798-31434da088d1.avro


The `files` table contains loads of information on data files, including column level statistics such as null counts, lower bounds, and upper bounds.

In [15]:
%%sql

SELECT file_path, file_format, record_count, null_value_counts, lower_bounds, upper_bounds
FROM taxis.files

file_path,file_format,record_count,null_value_counts,lower_bounds,upper_bounds
/home/iceberg/warehouse/taxis/data/00000-14-6ae850dc-1353-4747-b924-6416f7f703ff-00001.parquet,PARQUET,4953,"{1: 53, 2: 0, 3: 0, 4: 53, 5: 0, 6: 53, 7: 53, 8: 0, 9: 0, 10: 53, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0}","{1: bytearray(b'1'), 2: bytearray(b'2020-04-01 00:26'), 3: bytearray(b'2020-04-01 00:31'), 4: bytearray(b'0'), 5: bytearray(b'.01'), 6: bytearray(b'1'), 7: bytearray(b'N'), 8: bytearray(b'10'), 9: bytearray(b'100'), 10: bytearray(b'1'), 11: bytearray(b'-10.5'), 12: bytearray(b'-0.5'), 13: bytearray(b'-0.5'), 14: bytearray(b'-2.75'), 15: bytearray(b'0'), 16: bytearray(b'-0.3'), 17: bytearray(b'-10.3'), 18: bytearray(b'-2.5'), 19: bytearray(b'\x00\x80\xa2\xc5')}","{1: bytearray(b'2'), 2: bytearray(b'2020-04-30 23:27'), 3: bytearray(b'2020-04-30 23:32'), 4: bytearray(b'6'), 5: bytearray(b'2.00'), 6: bytearray(b'5'), 7: bytearray(b'Y'), 8: bytearray(b'97'), 9: bytearray(b'97'), 10: bytearray(b'4'), 11: bytearray(b'8'), 12: bytearray(b'3.5'), 13: bytearray(b'0.5'), 14: bytearray(b'9.8'), 15: bytearray(b'60'), 16: bytearray(b'0.3'), 17: bytearray(b'9.98'), 18: bytearray(b'2.5'), 19: bytearray(b'\x00\x00\x80@')}"


## Time Travel

The history table lists all snapshots and which parent snapshot they derive from. The `is_current_ancestor` flag let's you know if a snapshot is part of the linear history of the current snapshot of the table.

In [16]:
%%sql

SELECT *
FROM taxis.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2021-11-19 18:31:43.915000,5166552351258945488,None,True
2021-11-19 18:31:48.162000,4397789499468740865,5166552351258945488,True
2021-11-19 18:31:49.877000,7769498799260430115,4397789499468740865,True
2021-11-19 18:31:50.454000,6092828293901762509,7769498799260430115,True


You can time-travel by altering the `current-snapshot-id` property of the table to reference any snapshot in the table's history. Let's revert the table to it's original state by traveling to the very first snapshot ID.

In [17]:
%%sql --var df

SELECT *
FROM taxis.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2021-11-19 18:31:43.915000,5166552351258945488,None,True
2021-11-19 18:31:48.162000,4397789499468740865,5166552351258945488,True
2021-11-19 18:31:49.877000,7769498799260430115,4397789499468740865,True
2021-11-19 18:31:50.454000,6092828293901762509,7769498799260430115,True


In [18]:
original_snapshot = df.head().snapshot_id
print(original_snapshot)
spark.sql(f"ALTER TABLE taxis SET TBLPROPERTIES ('current-snapshot-id'={original_snapshot})")

5166552351258945488


DataFrame[]

In [19]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,fare,distance,fare_per_distance_unit
1,2020-04-01 00:41:22,2020-04-01 01:01:53,5.5,1.20,None
1,2020-04-01 00:56:00,2020-04-01 01:09:25,12.5,3.40,None
1,2020-04-01 00:00:26,2020-04-01 00:09:25,10,2.80,None
1,2020-04-01 00:24:38,2020-04-01 00:34:38,10,2.60,None
2,2020-04-01 00:13:24,2020-04-01 00:18:26,6.5,1.44,None
2,2020-04-01 00:24:36,2020-04-01 00:33:09,10.5,2.93,None
2,2020-04-01 00:56:56,2020-04-01 01:09:13,20,6.86,None
2,2020-04-01 00:06:56,2020-04-01 00:14:15,7,1.19,None
1,2020-04-01 00:50:05,2020-04-01 01:08:54,31.5,11.30,None
2,2020-04-01 00:07:10,2020-04-01 00:18:45,13,3.68,None


Another look at the history table shows that the original state of the table has been added as a new entry
with the original snapshot ID.

In [20]:
%%sql

SELECT *
FROM taxis.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2021-11-19 18:31:43.915000,5166552351258945488,None,True
2021-11-19 18:31:48.162000,4397789499468740865,5166552351258945488,False
2021-11-19 18:31:49.877000,7769498799260430115,4397789499468740865,False
2021-11-19 18:31:50.454000,6092828293901762509,7769498799260430115,False
2021-11-19 18:31:54.206000,5166552351258945488,None,True


In [21]:
%%sql

SELECT COUNT(*) as cnt
FROM taxis

cnt
237993
